@Author: Suresh <br>
@Date: 2024-09-04 <br>
@Last Modified by: Suresh <br>
@Last Modified: 2024-09-04 <br>
@Title :  To find out the infected population percentage locally and globally <br>

                                            INFECTED  POPULATION PERCENTAGE LOCALLY

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when

spark = SparkSession.builder.appName("Covid19IndiaAnalysis").getOrCreate()


file_path_india = r"file:///E:\microsoft-SQL\Covid Problem (1)\Covid Problem\covid_19_india.csv"
df = spark.read.csv(file_path_india, header=True, inferSchema=True)



df = df.withColumn("Confirmed", col("Confirmed").cast("int")) .withColumn("Cured", col("Cured").cast("int"))


result_df = (
    df
    .groupBy("State/UnionTerritory")
    .agg(
        sum("Confirmed").alias("Total_Confirmed"),
        sum("Cured").alias("Total_Cured")
    )
    .withColumn(
        "locally_Infected",
        when(col("Total_Confirmed") > 0,
             100 - (col("Total_Cured") * 100.0 / col("Total_Confirmed"))
        ).otherwise(None)
    )
)

result_df.printSchema()
result_df.show(truncate=False)


root
 |-- State/UnionTerritory: string (nullable = true)
 |-- Total_Confirmed: long (nullable = true)
 |-- Total_Cured: long (nullable = true)
 |-- locally_Infected: double (nullable = true)

+----------------------------------------+---------------+-----------+------------------+
|State/UnionTerritory                    |Total_Confirmed|Total_Cured|locally_Infected  |
+----------------------------------------+---------------+-----------+------------------+
|Nagaland                                |5041742        |4519526    |10.357848537271437|
|Karnataka                               |485970693      |441844360  |9.080039935659244 |
|Odisha                                  |160130533      |150923455  |5.749732938189865 |
|Kerala                                  |458906023      |420174235  |8.440026074793963 |
|Ladakh                                  |4054293        |3758960    |7.284451321105806 |
|Dadra and Nagar Haveli and Daman and Diu|1938632        |1841750    |4.997441494827285 

                                                INFECTED POPULATION PERCENTAGE GLOBALLY

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

spark = SparkSession.builder.appName("Covid19Analysis").getOrCreate()

file_path_india = r"file:///E:\microsoft-SQL\Covid Problem (1)\Covid Problem\covid_19_india.csv"
df = spark.read.csv(file_path_india, header=True, inferSchema=True)




result_df = df.groupBy("State/UnionTerritory") \
    .agg(
        sum("Confirmed").alias("Total_Cases"),
        sum("ConfirmedIndianNational").alias("Total_Active")
    ) \
    .withColumn(
        "Globally_Infected_percentage",
        (col("Total_Active") * 100.0) / col("Total_Cases")
    )



df.printSchema()
result_df.show()


root
 |-- Sno: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- State/UnionTerritory: string (nullable = true)
 |-- ConfirmedIndianNational: string (nullable = true)
 |-- ConfirmedForeignNational: string (nullable = true)
 |-- Cured: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Confirmed: integer (nullable = true)

+--------------------+-----------+------------+----------------------------+
|State/UnionTerritory|Total_Cases|Total_Active|Globally_Infected_percentage|
+--------------------+-----------+------------+----------------------------+
|            Nagaland|    5041742|        NULL|                        NULL|
|           Karnataka|  485970693|       405.0|        8.333835884214524E-5|
|              Odisha|  160130533|        23.0|        1.436328198570350...|
|              Kerala|  458906023|      1091.0|        2.377393072480986E-4|
|              Ladakh|    4054293|       162.0|         0.003995764